# Encode .PNGs -> .COE

In [179]:
from PIL import Image
import os
from collections import deque

def flood_fill_iterative(image, x, y, target_color, replacement_color):
    width, height = image.size
    pixel_data = image.load()
    queue = deque()
    queue.append((x, y))

    while queue:
        x, y = queue.popleft()
        current_color = pixel_data[x, y]
        if current_color == target_color:
            pixel_data[x, y] = replacement_color
            if x > 0: queue.append((x - 1, y))
            if x < width - 1: queue.append((x + 1, y))
            if y > 0: queue.append((x, y - 1))
            if y < height - 1: queue.append((x, y + 1))

def convert_image_to_coe(directory, coe_file_path, target_size=(56, 56)):
    with open(coe_file_path, 'w') as coe_file:
        coe_file.write("memory_initialization_radix=16;\n")
        coe_file.write("memory_initialization_vector=\n")

        filenames = [f for f in os.listdir(directory) if f.lower().endswith('.png')]
        sorted_filenames = sorted(filenames, key=lambda name: int(name.split('.')[0]))

        for filename in sorted_filenames:
            img_path = os.path.join(directory, filename)
            img = Image.open(img_path).convert('RGB')

            new_img = Image.new('RGB', target_size, "white")
            position = ((target_size[0] - img.width) // 2, (target_size[1] - img.height) // 2)
            new_img.paste(img, position)

            for x in range(target_size[0]):
                flood_fill_iterative(new_img, x, 0, (255, 255, 255), (254, 255, 255))  # Top edge
                flood_fill_iterative(new_img, x, target_size[1] - 1, (255, 255, 255), (254, 255, 255))  # Bottom edge
            for y in range(target_size[1]):
                flood_fill_iterative(new_img, 0, y, (255, 255, 255), (254, 255, 255))  # Left edge
                flood_fill_iterative(new_img, target_size[0] - 1, y, (255, 255, 255), (254, 255, 255))  # Right edge

            pixel_data = new_img.load()

            for y in range(target_size[1]):
                for x in range(target_size[0]):
                    r, g, b = pixel_data[x, y]
                    r >>= 4
                    g >>= 4
                    b >>= 4
                    # Write FFE for non-background white pixels, FFF for background white pixels
                    hex_value = f"{r:01X}{g:01X}{b:01X}"
                    if hex_value == 'FFF' and pixel_data[x, y] != (254, 255, 255):
                        hex_value = 'FFE'
                    coe_file.write(hex_value + ",\n")

            # Add a delimiter between sprites
            coe_file.write("000,\n")

        coe_file.write(";")

convert_image_to_coe('red-blue', 'pokemon_sprites.coe')

# Testing: Decode .COE -> PNG

In [181]:
from PIL import Image

def coe_to_png(coe_file, png_file, i, image_size=(56, 56)):
    # Read the COE file
    with open(coe_file, 'r') as file:
        lines = file.readlines()

    start_index = i * (image_size[0] * image_size[1] + 1)
    end_index = start_index + image_size[0] * image_size[1]

    pixel_values = []
    for line in lines[start_index + 2:end_index + 2]:
        parts = line.split(',')
        if len(parts) > 0:
            hex_value = parts[0].strip()
            if hex_value and len(hex_value) == 3:  # Ensure it's a 3-character hex string
                try:
                    red = int(hex_value[0], 16) * 17  # 17 is 255/15
                    green = int(hex_value[1], 16) * 17
                    blue = int(hex_value[2], 16) * 17
                    if hex_value == 'FFF':
                        red, green, blue = (255, 105, 180)  # Hot pink
                    pixel_values.append((red, green, blue))
                except ValueError:
                    continue  # Skip lines that don't contain valid hex values

    if len(pixel_values) < image_size[0] * image_size[1]:
        raise ValueError("Not enough pixel values for the i-th image in the COE file")

    image = Image.new('RGB', image_size)
    image.putdata(pixel_values)

    image.save(png_file)

coe_to_png('pokemon_sprites.coe', 'output_image.png', 10)